# Libraries

In [16]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import numpy as np
import scipy
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns
from collections import Counter
import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import operator

import nltk
from nltk.corpus import stopwords
import torch
from torchtext import data
from torchtext import datasets

In [18]:
! pip install transformers -q
from transformers import pipeline

In [19]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


# Recommendations from Keywords

## With Emotions

In [20]:
def cosine_similarity_of(text1, text2):
        
        # taking words
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        # creating dictionary from each word and count
        vector1 = Counter(first)
        vector2 = Counter(second)

        # changing vectors to sets to find comman words with intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            # getting amount of each common word for vectors and multiply them also add them
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        # getting summation of word counts for every vector
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        # calculating magnitude of vector with squared sums
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude

In [21]:
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/To Do Reviews/activities_for_keywords_recommendation.csv', index_col = 0) 

In [22]:
df

,Name,ReviewTitle,Review,EmotionFromTitle,Review_Clean,Title_Clean,Reviewer_Count,Activity_Count,word_without_stop
UserId,,,,,,,,,
1,acropolis,acropolis - breathtaking,what an experience. i highly recommend doing t...,admiration,what an experience i highly recommend doing th...,acropolis breathtaking,11,1500,acropolis breathtaking
662,mount lycabettus,one of the top places to view athens,"incredible views, but it's not a must-see unle...",admiration,incredible views but its not a mustsee unless ...,one of the top places to view athens,11,1250,one top places view athens
660,mount lycabettus,give it a miss honestly! not worth it!,this place wasn't on my list it was only until...,disapproval,this place wasnt on my list it was only until ...,give it a miss honestly not worth it,11,1250,give miss honestly worth
659,mount lycabettus,nice view,"we went to lycabettus hill by foot, it is a go...",admiration,we went to lycabettus hill by foot it is a goo...,nice view,11,1250,nice view
658,mount lycabettus,great hike with a stunning view of athens,"it is worth it to hike up the hill, when you g...",admiration,it is worth it to hike up the hill when you go...,great hike with a stunning view of athens,11,1250,great hike stunning view athens
...,...,...,...,...,...,...,...,...,...
1478,acropolis,don't miss this,you would be forgiven for thinking you could s...,caring,you would be forgiven for thinking you could s...,dont miss this,1,1500,dont miss
1479,acropolis,"obviously a ""must see""","impressed to be in this world heritage site, a...",approval,impressed to be in this world heritage site al...,obviously a must see,1,1500,obviously must see
1480,acropolis,magical acropolis,i visit the acropolis every time i come to ath...,admiration,i visit the acropolis every time i come to ath...,magical acropolis,1,1500,magical acropolis


In [29]:
def get_recommendations_with_emotion(df,keywords):

        score_dict = {}

        emotion_from_keywords = emotion(keywords)[0]['label']
        print('Emotion is ' + emotion_from_keywords)

        for index, row in df.iterrows():
            score_from_similarity = cosine_similarity_of(str(row['word_without_stop']), keywords)

            emotion_from_comment = row['EmotionFromTitle']

            # Αν το συναίσθημα από τα keywords ισοδυναμεί με το συναίσθημα του σχολίου τότε το score αυξάνεται κατά 10%
            if(emotion_from_comment == emotion_from_keywords):
              last_score = score_from_similarity + 0.2*score_from_similarity
            else:
              last_score = score_from_similarity

            score_dict[index] = last_score
            
        #sort τα σχόλια
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        #δημιουργία νέου dataframe για την αποθήκευση των recommendations
        result = pd.DataFrame(columns=('Name', 'score'))

        # τα reviews με τα υψηλότερα cosign similarity
        for i in sorted_scores:
            result = result.append({'Name': df.iloc[i[0]]['Name'],
                                        'score': i[1]}, ignore_index=True)
            
            counter += 1

            if counter>20:
                break

        result = result.drop_duplicates(subset=['Name'], keep='first')
        result = result.reset_index(drop=True)   
        return result

In [32]:
keywords = "unforgettable experience"
recommendations = get_recommendations_with_emotion(df,keywords)
recommendations[:15]

Emotion is fear


,Name,score
0,monastiraki,0.600000
1,parthenon,0.500000
2,mount lycabettus,0.408248
3,national archaeological museum,0.267261
